In [1]:
import os
import numpy as np
import pandas as pd
import napari
from napari import Viewer
from magicgui import magicgui
from tifffile import imread,imwrite,TiffFile
from skimage.io import imsave
from skimage.io import imread as imread_skimage

In [2]:
df_path = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\230117_exp_summary.pkl'

save_path = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\example_images'

In [3]:
df = pd.read_pickle(df_path)
df

,file_name,exp_dir,im_path,slide,well,exp,marker,dye,im_round,density,donor,treatment
0,T3737_9_2_1.0.4_R000_Cy3_CycB1-AF555_FINAL_AFR...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CycB1,Cy3,1,150,D1,0
1,T3737_9_2_1.0.4_R000_Cy5_CD8a-AF647_FINAL_AFR_...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CD8a,Cy5,1,150,D1,0
2,T3737_9_2_1.0.4_R000_Cy7_CD44-AF750_FINAL_AFR_...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CD44,Cy7,1,150,D1,0
3,T3737_9_2_1.0.4_R000_DAPI__FINAL_F.tif,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_D...,T3737_9_2,R000,T3737_9_2_R000,DNA,DAPI,1,150,D1,0
4,T3737_9_2_1.0.4_R000_FITC_PCNA-AF488_FINAL_AFR...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_F...,T3737_9_2,R000,T3737_9_2_R000,PCNA,FITC,1,150,D1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
155,T3937_39_4.0.4_R007_Cy3_p21-AF555_FINAL_AFR_F.tif,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy3...,T3937_39,R007,T3937_39_R007,p21,Cy3,4,200,D2,24
156,T3937_39_4.0.4_R007_Cy5_FoxP3-AF647_FINAL_AFR_...,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy5...,T3937_39,R007,T3937_39_R007,FoxP3,Cy5,4,200,D2,24
157,T3937_39_4.0.4_R007_Cy7_cycB2-AF750_FINAL_AFR_...,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy7...,T3937_39,R007,T3937_39_R007,cycB2,Cy7,4,200,D2,24
158,T3937_39_4.0.4_R007_DAPI__FINAL_F.tif,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_DAP...,T3937_39,R007,T3937_39_R007,DNA,DAPI,4,200,D2,24


In [16]:
def save_selected_region(viewer: Viewer):

    # read the info about the regions
    fovs = viewer.layers['Regions'].data

    i = 0 
    for fov in fovs:

        fov = fov.astype(int)

        small_stack = im_stack[:,fov[0,0]:fov[2,0],fov[0,1]:fov[2,1]]

        for small_im,im_name in zip(small_stack,df_sel.marker):

            ch_path = os.path.join(save_path,sel_exp)
            ch_name = f'{sel_exp}_{im_name}_region_{str(i).zfill(2)}.tif'

            if not(os.path.isdir(ch_path)):

                os.mkdir(ch_path)
                ch_path = os.path.join(ch_path,f'region_{str(i).zfill(2)}')
                os.mkdir(ch_path)

            else:

                ch_path = os.path.join(ch_path,f'region_{str(i).zfill(2)}')

            imwrite(os.path.join(ch_path,ch_name),small_im)

        i = i+1
    
    viewer.status = 'Region has been saved.'

In [22]:
sel_exp = 'T3937_39_R000'

df_sel = df.loc[df.exp==sel_exp,:]
len(df_sel)

20

In [23]:
# load all the channels from the selected experiment

im_stack = []
for ind,sel_im in df_sel.iterrows():

    im_path = sel_im.im_path
    im = imread(im_path)

    im_stack.append(im)

im_stack = np.array(im_stack)
im_stack.shape

(20, 22400, 13251)

In [24]:
# calculate starting square

row_num = 1000
col_num = 1000

fov = np.array([[int((im_stack.shape[1]/2) - (row_num/2)),int((im_stack.shape[2]/2) - (col_num/2))],
                [int((im_stack.shape[1]/2) - (row_num/2)),int((im_stack.shape[2]/2) + (col_num/2))],
                [int((im_stack.shape[1]/2) + (row_num/2)),int((im_stack.shape[2]/2) + (col_num/2))],
                [int((im_stack.shape[1]/2) + (row_num/2)),int((im_stack.shape[2]/2) - (col_num/2))]])


In [26]:
# display napari viewer
viewer = napari.Viewer()
viewer.add_image(im_stack)
viewer.add_shapes(data=fov,name='Regions')

# add saving button to napari
save_data = magicgui(save_selected_region, call_button='Save Region')
viewer.window.add_dock_widget(save_data,area='left')